In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [6]:
glove_wiki_vectors.most_similar("cat")

[('dog', 0.8798074126243591),
 ('rabbit', 0.7424427270889282),
 ('cats', 0.732300341129303),
 ('monkey', 0.7288709878921509),
 ('pet', 0.7190139889717102),
 ('dogs', 0.7163873314857483),
 ('mouse', 0.6915250420570374),
 ('puppy', 0.6800068616867065),
 ('rat', 0.6641027331352234),
 ('spider', 0.6501135230064392)]

In [7]:
glove_wiki_vectors.most_similar("seafood")

[('shrimp', 0.7414492964744568),
 ('shellfish', 0.728592574596405),
 ('meat', 0.7124432921409607),
 ('tuna', 0.7002078294754028),
 ('fish', 0.6940563321113586),
 ('dishes', 0.6856633424758911),
 ('sushi', 0.6691842675209045),
 ('oysters', 0.6684554219245911),
 ('chicken', 0.6671671271324158),
 ('lobster', 0.6669976711273193)]

In [8]:
glove_wiki_vectors.most_similar("exams")

[('exam', 0.8966719508171082),
 ('examinations', 0.8652294278144836),
 ('coursework', 0.698704183101654),
 ('examination', 0.6971177458763123),
 ('graduation', 0.6346420049667358),
 ('evaluations', 0.631233811378479),
 ('math', 0.618693470954895),
 ('courses', 0.618667483329773),
 ('placement', 0.612671434879303),
 ('a-level', 0.611441433429718)]

In [9]:
glove_wiki_vectors.most_similar("christmas")

[('holiday', 0.8736870288848877),
 ('thanksgiving', 0.7960303425788879),
 ('easter', 0.7859223484992981),
 ('eve', 0.7467230558395386),
 ('halloween', 0.7238508462905884),
 ('holidays', 0.7132129669189453),
 ('wedding', 0.688292920589447),
 ('day', 0.6734689474105835),
 ('birthday', 0.6719172596931458),
 ('celebration', 0.6699538826942444)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [10]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [11]:
for pair in word_pairs:
    print(
        "The similarity between %s and %s is %0.3f"
        % (pair[0], pair[1], glove_wiki_vectors.similarity(pair[0], pair[1]))
    )


The similarity between coast and shore is 0.700
The similarity between clothes and closet is 0.546
The similarity between old and new is 0.643
The similarity between smart and intelligent is 0.755
The similarity between dog and cat is 0.880
The similarity between tree and lawyer is 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.  

In [12]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [13]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [14]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [15]:
glove_wiki_vectors.similarity("black", "rich")

0.5174519

<div class="alert alert-warning">

Solution_1.3
    
</div>

There are definitely some worrisome biases in that doctors are often men, and nurses are often women. In the analogy list above, we can also see "pregnant" and "mother" which infer that women are only really in the hospital to give birth or when they are pregnant.

In [16]:
analogy("man", "manager", "woman")

man : manager :: woman : ?


,Analogy word,Score
0,assistant,0.656641
1,job,0.626392
2,supervisor,0.593479
3,owner,0.587832
4,hired,0.577390
5,consultant,0.570659
6,director,0.565653
7,office,0.564568
8,employee,0.561287
9,ceo,0.556422


The highest similarity is ASSISTANT?? I'm fuming.

In [17]:
analogy("woman", "manager", "man")

woman : manager :: man : ?


,Analogy word,Score
0,boss,0.712273
1,coach,0.642365
2,owner,0.631863
3,managing,0.630722
4,mike,0.614558
5,rangers,0.614265
6,ferguson,0.603462
7,team,0.601174
8,management,0.590261
9,steve,0.589599


When we swap the roles, the highest for "manager" for a man is BOSS. This shows the clear bias/stereotype.

In [18]:
analogy("man", "secretary", "woman")

man : secretary :: woman : ?


,Analogy word,Score
0,affairs,0.648443
1,deputy,0.648281
2,representative,0.630497
3,spokeswoman,0.624199
4,spokesperson,0.612689
5,albright,0.604644
6,ambassador,0.600308
7,department,0.592507
8,minister,0.590829
9,office,0.589634


In [19]:
analogy("woman", "secretary", "man")

woman : secretary :: man : ?


,Analogy word,Score
0,general,0.736190
1,chief,0.692401
2,deputy,0.671780
3,chairman,0.664504
4,rumsfeld,0.653890
5,donald,0.635758
6,spokesman,0.628681
7,vice,0.618626
8,president,0.618278
9,minister,0.616832


Another example when a woman is a "secretary", the man is a "general", where if a man is a "secretary", the woman is related to "affairs"

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

1. Overall, the quality of the pre-trained embeddings are fine, but when it comes to gender-focused embeddings, it is especially sub-par. For example, when using the analogy relating a male or female to a manager, the woman's highest similarity is an assistant while the man's highest similarity is a boss due to the biases made from the data that the model trained on.

2. One example we could think of how such a model could cause harm is if we were to use an AI occuptation prediction service. Let's say this service provides high school students with occupational options by taking in their age, gender, and other interests (i.e. hobbies). By using a gender stereotyped model (where word embeddings have not accounted for gender biases), this AI occupation prediction service might base its occupation prediction on gender stereotypes (i.e. women as nurses and men as CEOs). Gender stereotypes are harmful in that they assume roles for certain genders, more specifically, men and women. However, with the constant change in society and advocacy for gender equality, the social roles associated with men and women has drastically changed overtime. To associate people with past gender roles can be unfair and may actually set back the efforts made to promote equality and even reinforce gender stereotypes.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Classification with pre-trained embeddings 
rubric={points:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`) and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. 
2. Train logistic regression with average embedding representation extracted using spaCy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [20]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [21]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [22]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [23]:
import spacy

nlp = spacy.load("en_core_web_md")

C:\Users\User\miniconda3\envs\cpsc330\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<div class="alert alert-warning">

Solution_1.5
    
</div>

In [24]:
pipe_lr = make_pipeline(
    CountVectorizer(stop_words="english"),
    LogisticRegression(max_iter=1000),
)
pipe_lr.fit(X_train, y_train)
feature_names = np.array(pipe_lr.named_steps["countvectorizer"].get_feature_names_out())
coeffs = pipe_lr.named_steps["logisticregression"].coef_.flatten()
predictions = pipe_lr.predict(X_test)
predictions

array(['achievement', 'affection', 'bonding', ..., 'affection',
       'achievement', 'bonding'], dtype=object)

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions, target_names=y_test.unique()))

                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
          nature       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
        exercise       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



In [26]:
pipe = make_pipeline(
    CountVectorizer(stop_words="english"), 
    LogisticRegression(max_iter=1000)
)
pipe.named_steps["countvectorizer"].fit(X_train)
X_train_transformed = pipe.named_steps["countvectorizer"].transform(X_train)
pipe.fit(X_train, y_train)

X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

lgr = LogisticRegression(max_iter=10000)
lgr.fit(X_train_embeddings, y_train)

LogisticRegression(max_iter=10000)

In [27]:
predictions = lgr.predict(X_test_embeddings)
predictions

array(['achievement', 'affection', 'bonding', ..., 'affection',
       'achievement', 'affection'], dtype=object)

In [28]:
print(classification_report(y_test, predictions, target_names=y_test.unique()))

                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.57      0.51      0.54       469
          nature       0.68      0.76      0.72        74
         leisure       0.72      0.65      0.68       407
        exercise       0.68      0.72      0.70        71

        accuracy                           0.79      4238
       macro avg       0.74      0.74      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. CountVectorizer uses only information from your text; average-embedding uses information from outside sources (pre-trained embeddings). CountVectorizer returns a sparse matrix while average-embedding returns dense vectors. CountVectorizer also takes in the frequency of each word while average-embedding applies a weight to each word
2. CountVectorizer, because each number directly corresponds to something real (i.e. the counts of the words). In average-embedding, the coefficients are not very interpretable. 
3. Yes, we are using transfer learning when we use the spacy pre-trained model. I'm not sure if we can conclusively say we observe benefits of transfer-learning, since we don't have a base-line CountVectorizer model to compare it to. The other model was also pre-trained. But the spacy model seems to perform better than the glove_wiki model.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [29]:
from sklearn.datasets import fetch_20newsgroups

In [30]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Type your answer here, replacing this text._

In [31]:
# code adapted from Lecture 18
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "AUX", "ADP", "SPACE", "X", "SYM"],
):
    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)



In [32]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. 922 sence wonder help u.s.c. ...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,following problem bug appreciate help create w...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq part3 modify 1993/04/04 subject pa...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference feds mandate l...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway 4dx-33v try configure x11r5 need c...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

As part of our tokenization, we decided that a token should be at least two characters long and should not belong to the parts of speech stated in "irrelevant_pos" (i.e. token should not be punctuation, space, etc.). Then, we iterate through each token and if it meets these requirements, we take the lemma of the word. We decided to take out any adverbs, pronouns, conjunctions, punctuation, particles, determiners, auxiliaries, adpositions, spaces and any symbols or emojis since all of these are part of our everyday speech and grammar but do not actually add any meaning for the model and will not help in any way when classifying into different topics etc. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

In [33]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = len(newsgroups_train.target_names) #8 topics
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)


We chose the number of unique categories in the dataset that we are considering (which is 8) as the number of topics because that is how many categories we want to consider to categorize our data into.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

In [34]:
vec = CountVectorizer(stop_words="english")
vec_X = vec.fit_transform(df["text"])
lda.fit(vec_X) 
document_topics = lda.transform(vec_X)

In [35]:
import mglearn
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

mglearn.tools.print_topics(
    topics=range(8),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
jpeg          armenian      people        jesus         team          
00            turkish       gun           god           game          
gif           israel        said          25            year          
02            jews          don           entry         play          
03            armenians     know          christ        season        
image         israeli       just          lord          games         
01            people        like          son           hockey        
color         turkey        didn          father        players       
04            10            file          sin           good          
images        government    guns          entries       league        


topic 5       topic 6       topic 7       
--------      --------      --------      
god           file          edu           
people        out

Topic 0: comp.graphcs 

Topic 1: talk.politics.mideast

Topic 2: talk.politics.guns

Topic 3: soc.religion.christian

Topic 4: rec.sport.hockey

Topic 5: alt.atheism

Topic 6: comp.windows.x

Topic 7: comp.graphcs

I have categorized the topics into the categories given at the beginning of this question, but it is strange how there are two topics that seem to be comp.graphics and rec.sport.baseball is nowhere to be found. 

In [36]:
cats

['rec.sport.hockey',
 'rec.sport.baseball',
 'soc.religion.christian',
 'alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'talk.politics.mideast',
 'talk.politics.guns']

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

In [37]:
document_topics[0], document_topics[1], document_topics[2], document_topics[3], document_topics[4]

(array([0.0017858 , 0.00178749, 0.69669791, 0.00178697, 0.00178701,
        0.15164288, 0.00178652, 0.14272542]),
 array([0.00162432, 0.00162489, 0.00162532, 0.00162433, 0.28949989,
        0.07820187, 0.00162457, 0.62417482]),
 array([0.00186689, 0.0018664 , 0.00186779, 0.11652469, 0.0018671 ,
        0.60612639, 0.00186622, 0.26801451]),
 array([0.00320824, 0.00320685, 0.00320957, 0.00320879, 0.00320766,
        0.00321727, 0.00320527, 0.97753634]),
 array([0.00271819, 0.68047257, 0.00272123, 0.30320566, 0.00271887,
        0.0027228 , 0.00271944, 0.00272124]))

Our document topic assignments show how each topic within each document is made up of. For example, as seen above, document 1 has pretty much 69% topic 3, 15% topic 6, and negligent percentages of the other topics. It seems like each document seems to have an overpowering topic with all the other topics being quite close to 0%. 

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

1. The assumption we’re making here is that we don’t need to explain how the binary classification works and we only need to explain how the models extend to the multiclass case. 

    Decision Tree - a decision tree for binary and multiclass classification is quite similar. At each node, the stem splits into two based on a condition; this splitting continues until an end condition is reached (e.g. depth is max_depth). 

    KNN - exactly the same as binary classification: use the k nearest neighbors to the point to determine the class of the point by majority vote

    Random Forest - this is essentially an ensemble of decision trees so if decision trees can handle multi-class classification then it will work too. It takes a vote from each of the tree. 

    Logistic Regression - two options: one-versus-rest, and cross-entropy loss. One-versus-rest is essentially a series of binary classifications. I don’t know if we learned cross-entropy loss. 

    SVM RBF - two modes in sci-kit learn: one-versus-one and one-versus-rest. One-versus-one does binary comparisons between each class one at a time (computationally expensive!). One-versus-rest makes binary comparisons between one class and the rest of the classes. 

2. Transfer learning in NLP is using models that are trained on other data, like pre-trained word embeddings, to aid in our own tasks.

3. This property is useful because we can extract the layers for our own use; the models are “dissectable”. It is useful, for example, to extract features from a classification model by extracting the last layer before the classification. We can train another model (e.g. logistic regression) on these features. This allows us to use pre-trained models even if our classes aren’t exactly the classes the model was trained on!


<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)